In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1)
import sys
import sklearn
import sklearn.ensemble
from anchor import utils
from anchor import anchor_tabular

In [2]:
# make sure you have adult/adult.data inside dataset_folder
dataset_folder = '../data/'
dataset = utils.load_dataset('adult', balance=True, dataset_folder=dataset_folder, discretize=True)

In [3]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=50, n_jobs=5)
rf.fit(dataset.train, dataset.labels_train)
print('Train', sklearn.metrics.accuracy_score(dataset.labels_train, rf.predict(dataset.train)))
print('Test', sklearn.metrics.accuracy_score(dataset.labels_test, rf.predict(dataset.test)))

Train 0.9350338780390594
Test 0.8489483747609943


--------------
# Usage

In [4]:
import xaibenchmark as xb

In [5]:
class AnchorsExplainer(xb.Explainer):
    
    def __init__(self, class_names, feature_names, train, categorical_names):
        
        self.explainer = anchor_tabular.AnchorTabularExplainer(
            class_names,
            feature_names,
            train,
            categorical_names)
        self.train = train
        
    def explain_instance(self, instance, predictor, threshold=0.95):
        self.explanation = self.explainer.explain_instance(instance, predictor, threshold=threshold)
        self.instance = instance
        return self.explanation
    
    @xb.metric
    def coverage(self):
        if hasattr(self, 'explanation'):
            return self.explanation.coverage()
        
    
    @xb.metric
    def precision(self):
        if hasattr(self, 'explanation'):
            return self.explanation.precision()
        
    @xb.utility
    def get_neighborhood_instances(self): 
        if hasattr(self, 'explanation'):
            fit_anchor = np.where(np.all(self.train[:, self.explanation.features()] == self.instance[self.explanation.features()], axis=1))[0]
            return self.train[fit_anchor]
        return []
    
    @xb.utility
    def get_explained_instance(self):
        return self.instance
    
    @xb.utility
    def distance(self, x, y):
        return np.linalg.norm(x-y)

In [6]:
# instantiate anchors explainer
exp = AnchorsExplainer(dataset.class_names,
    dataset.feature_names,
    dataset.train,
    dataset.categorical_names)

In [7]:
# get all currently defined metrics
exp.metrics()

{'coverage', 'precision'}

In [8]:
# report all current metrics
exp.report()

{('coverage', nan), ('precision', nan)}

In [9]:
exp.get_neighborhood_instances()

[]

In [10]:
# explain a single instance (needed to compute coverage)
explanation = exp.explain_instance(dataset.test[0], rf.predict, threshold=0.95)

In [11]:
# get all currently defined metrics
exp.metrics()

{'coverage', 'precision'}

In [12]:
# report all current metrics
exp.report()

{('coverage', 0.0161), ('precision', 0.9833333333333333)}

In [13]:
# infer other possible metrics
exp.infer_metrics()

inferred metrics: {'furthest_distance', 'inverse_coverage', 'precision', 'coverage'}


In [14]:
# get all currently defined metrics
exp.metrics()

{'coverage', 'furthest_distance', 'inverse_coverage', 'precision'}

In [15]:
# report all current metrics
exp.report()

AttributeError: 'AnchorsExplainer' object has no attribute 'get_neigborhood_instances'